# Assignment 3 

### Importing Libraries

In [3]:
import matplotlib
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import csv

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Preparing Data

In [13]:
# Reading the data
train_file = open('tel_train.csv', encoding='utf-8')
test_file = open('tel_test.csv', encoding='utf-8')
val_file = open('tel_valid.csv', encoding='utf-8')

# Get pairs of (English, Telugu) words
def read_data(file):
    lines = file.readlines()
    pairs = [[s for s in l.split(',')] for l in lines]
    return pairs

train_pairs = read_data(train_file)
test_pairs = read_data(test_file)
val_pairs = read_data(val_file)





['mathaadhikaarulatho', 'మతాధికారులతో\n']


### The Encoder

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

### The Decoder

In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)